## Imports and loading data

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
image_size=(256,256)
dir="data/padded-resized/"
dataset=tf.keras.preprocessing.image_dataset_from_directory(dir+"train",batch_size=32,image_size=image_size,
                                                            validation_split=0.1,
                                                            subset="training",
                                                            seed=123,
                                                            shuffle=True,color_mode="rgb")
test_dataset=tf.keras.preprocessing.image_dataset_from_directory(dir+"train",batch_size=32,image_size=image_size,
                                                            validation_split=0.1,
                                                            seed=123,
                                                            subset="validation",
                                                            shuffle=True,color_mode="rgb")



## Loading VGG16 model

In [ ]:
b=dataset.take(1)
for i,l in b:
    print(l)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

#????base_model=InceptionV3(weights="imagenet",include_top=False,input_shape=(256,256,3))
#base_model = VGG16(weights="imagenet", include_top=False, input_shape=(256,256,3))
#base_model = ResNet101(weights="imagenet", include_top=False, input_shape=(256,256,3))
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(256,256,3))
#base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(256,256,3))

base_model.trainable = False


In [ ]:
base_model.summary()

## Adding Layers

In [ ]:
from tensorflow.keras import layers, models


flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(1024, activation='relu')
#dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(9, activation='softmax')



model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
 #   dense_layer_2,
    prediction_layer
])


In [ ]:
model.summary()

## Training model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

history=model.fit(dataset, validation_data=test_dataset,epochs=3, batch_size=32, callbacks=[es])

In [ ]:
print(history.history.keys())
print(history.history['val_loss'])


In [ ]:
model.evaluate(test_dataset)

In [ ]:
imgs=[]
labels=[]
test_dataset=test_dataset.unbatch()
for x,y in test_dataset.as_numpy_iterator(): 
    imgs.append(x)
    labels.append(y)
imgs=np.array(imgs)
labels=np.array(labels)

In [ ]:


probabilities=model.predict(imgs)
predictions=np.argmax(probabilities,axis=1)


In [ ]:

m=tf.math.confusion_matrix(labels,predictions)
m

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 7))
sb.heatmap(m/np.sum(m,axis=1).reshape(9,1), xticklabels=dataset.class_names, yticklabels=dataset.class_names, 
            annot=True,fmt=".2f")
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()